#### last updated June 19 2025

##### Read `ITIS.sqlite` from https://www.itis.gov/downloads/
- The itis.sqlite file is the SQLite database version of the Integrated Taxonomic Information System (ITIS). It contains standardized scientific and common names for plants, animals, fungi, and microbes.

| Table Name          | Description                                                               |
| ------------------- | ------------------------------------------------------------------------- |
| `taxonomic_units`   | Core table with TSN (Taxonomic Serial Number), scientific name, and rank. |
| `vernaculars`       | Common names associated with TSNs (e.g., "walleye" → TSN 168507).         |
| `hierarchy`         | Parent-child relationships (e.g., genus to species).                      |
| `synonym_links`     | Links between valid TSNs and their synonyms.                              |
| `taxon_authors_lkp` | Authorship info for scientific names.                                     |
| `kingdoms`          | List of biological kingdoms (e.g., Animalia, Plantae).                    |

  

#### Import libraries

In [3]:
import pandas as pd

import numpy as np
import sqlalchemy as sql
import pandas as pd
import sqlite3
pd.set_option("display.max_columns", 100)
# pd.set_option("display.latex.repr", False)
pd.set_option('display.max_colwidth', 100)

import thefuzz
from thefuzz import (fuzz, process)

#### Read the Table names from the database

In [5]:
# make a connection object to ITIS.sqlite
conn = sqlite3.connect("ITIS.sqlite")
# Make a cursor to run SQL commands
cursor = conn.cursor()                 
### SQL to read the Table Names from the database
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor.execute(sql_query)
## print Table name
print(cursor.fetchall())

[('HierarchyToRank',), ('change_comments',), ('change_operations',), ('change_tracks',), ('chg_operation_lkp',), ('comments',), ('experts',), ('geographic_div',), ('hierarchy',), ('jurisdiction',), ('kingdoms',), ('longnames',), ('nodc_ids',), ('other_sources',), ('publications',), ('reference_links',), ('reviews',), ('strippedauthor',), ('synonym_links',), ('taxon_authors_lkp',), ('taxon_unit_types',), ('taxonomic_units',), ('tu_comments_links',), ('vern_ref_links',), ('vernaculars',)]


#### Read `kingdoms` Table

In [7]:
df_kingdoms = pd.read_sql("SELECT * FROM kingdoms", con=conn)

In [8]:
df_kingdoms.head(10)

,kingdom_id,kingdom_name,update_date
0,1,Bacteria,2014-08-20
1,2,Protozoa,2004-06-04
2,3,Plantae,1996-03-26
3,4,Fungi,1996-03-26
4,5,Animalia,1996-03-26
5,6,Chromista,2004-06-04
6,7,Archaea,2014-08-20


In [9]:
df_kingdoms["kingdom_name"].unique()

array(['Bacteria', 'Protozoa', 'Plantae', 'Fungi', 'Animalia',
       'Chromista', 'Archaea'], dtype=object)

#### Read Table `taxonomic_units` - has 965,032 rows, 26 cols 

In [11]:
df = pd.read_sql("SELECT * FROM taxonomic_units", con=conn)

In [12]:
df.head()

,tsn,unit_ind1,unit_name1,unit_ind2,unit_name2,unit_ind3,unit_name3,unit_ind4,unit_name4,unnamed_taxon_ind,name_usage,unaccept_reason,credibility_rtng,completeness_rtng,currency_rating,phylo_sort_seq,initial_time_stamp,parent_tsn,taxon_author_id,hybrid_author_id,kingdom_id,rank_id,update_date,uncertain_prnt_ind,n_usage,complete_name
0,50,None,Bacteria,None,None,None,None,None,None,None,valid,None,TWG standards met,partial,2012,0,1996-06-13 14:51:08,0.0,184733,0,1,10,2014-08-20,None,valid,Bacteria
1,51,None,Schizomycetes,None,None,None,None,None,None,None,invalid,"unavailable, database artifact",Minimum taxonomic/nomenclature review,unknown,unknown,0,1996-06-13 14:51:08,0.0,0,0,1,60,2015-03-02,No,invalid,Schizomycetes
2,52,None,Archangiaceae,None,None,None,None,None,None,None,invalid,"unavailable, database artifact",Minimum taxonomic/nomenclature review,unknown,unknown,0,1996-06-13 14:51:08,0.0,0,0,1,140,2015-03-02,No,invalid,Archangiaceae
3,53,None,Pseudomonadales,None,None,None,None,None,None,None,valid,None,TWG standards met,complete,2012,0,1996-06-13 14:51:08,956156.0,184799,0,1,100,2015-03-02,No,valid,Pseudomonadales
4,54,None,Rhodobacteriineae,None,None,None,None,None,None,None,invalid,"unavailable, database artifact",Minimum taxonomic/nomenclature review,unknown,unknown,0,1996-06-13 14:51:08,0.0,0,0,1,110,2015-03-02,No,invalid,Rhodobacteriineae


In [13]:
print("\n Shape of table")
print(df.shape)


print("\n Column Name")
print(df.columns)


 Shape of table
(965032, 26)

 Column Name
Index(['tsn', 'unit_ind1', 'unit_name1', 'unit_ind2', 'unit_name2',
       'unit_ind3', 'unit_name3', 'unit_ind4', 'unit_name4',
       'unnamed_taxon_ind', 'name_usage', 'unaccept_reason',
       'credibility_rtng', 'completeness_rtng', 'currency_rating',
       'phylo_sort_seq', 'initial_time_stamp', 'parent_tsn', 'taxon_author_id',
       'hybrid_author_id', 'kingdom_id', 'rank_id', 'update_date',
       'uncertain_prnt_ind', 'n_usage', 'complete_name'],
      dtype='object')


In [14]:
df_completename = df[['tsn', 'complete_name', 'n_usage', 'kingdom_id']]

In [15]:
df_completename.head()

,tsn,complete_name,n_usage,kingdom_id
0,50,Bacteria,valid,1
1,51,Schizomycetes,invalid,1
2,52,Archangiaceae,invalid,1
3,53,Pseudomonadales,valid,1
4,54,Rhodobacteriineae,invalid,1


#### Take only 'Animalia' kingdom_id = 5 and valid n_usage

In [17]:
df_completename_1 = df_completename[(df_completename["kingdom_id"]==5) & \
                                        (df_completename["n_usage"]=="valid")]
df_completename_1.shape

(555024, 4)

In [18]:
df_completename_1.head()

,tsn,complete_name,n_usage,kingdom_id
13707,13757,Antilocapra americana peninsularis,valid,5
18110,18161,Hornera smitti,valid,5
18111,18162,Hornera falklandica,valid,5
18112,18163,Hornera antarctica,valid,5
18113,18164,Hornera americana,valid,5


#### In ITIS table `vernaculars` names are common names

In [20]:
df2 = pd.read_sql("SELECT * FROM vernaculars", con=conn)

In [21]:
df2.head()

,tsn,vernacular_name,language,approved_ind,update_date,vern_id
0,50,bactéries,French,N,2015-03-02,159941
1,50,bacteria,English,N,2015-03-02,159942
2,50,bacterias,Spanish,N,2015-03-02,159943
3,50,bactérias,Portuguese,N,2015-03-02,159944
4,601,cyanophytes,French,N,2003-05-21,85559


In [22]:
# df2[df2["tsn"]==161706]

In [23]:
df2.shape

(152250, 6)

#### Common names are in many different languages

In [25]:
df2["language"].unique()

array(['French', 'English', 'Spanish', 'Portuguese', 'unspecified',
       'Hawaiian', 'Afrikaans', 'German', 'Native American', 'Finnish',
       'Swedish', 'Danish', 'Fijan', 'Italian', 'Japanese', 'Khmer',
       'Arabic', 'Icelandic', 'Iglulik Inuit', 'Chinese', 'Hindi',
       'Estonian', 'Turkish', 'Dutch', 'Norwegian', 'Indonesian',
       'Manyika', 'Ndau', 'Shona', 'Hausa', 'Greek', 'Djuka', 'Galibi',
       'Lao', 'Javanese', 'Korean', 'Czech', 'Hungarian', 'eng',
       'Australian', 'Northern Sotho', 'Swati', 'Xhosa', 'Zulu',
       'Malagasy', 'Bengali', 'Romanian', 'Unspecified', 'Fijian',
       'Slovak', 'Croatian'], dtype=object)

In [26]:
df2 = df2[df2["language"]=="English"]
df2.shape

(115313, 6)

#### merge: inner join on tsn column

In [28]:
df_merge = pd.merge(df_completename_1, df2[["tsn", "vernacular_name", "language"]], on="tsn", how="inner")

In [29]:
df_merge.shape

(59299, 6)

In [30]:
df_merge.head()

,tsn,complete_name,n_usage,kingdom_id,vernacular_name,language
0,13757,Antilocapra americana peninsularis,valid,5,peninsular pronghorn,English
1,46861,Porifera,valid,5,sponges,English
2,47618,Hippospongia lachne,valid,5,sheepswool sponge,English
3,48738,Cnidaria,valid,5,cnidarians,English
4,48738,Cnidaria,valid,5,coelenterates,English


#### Take only valid name

In [32]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59299 entries, 0 to 59298
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tsn              59299 non-null  int64 
 1   complete_name    59299 non-null  object
 2   n_usage          59299 non-null  object
 3   kingdom_id       59299 non-null  int64 
 4   vernacular_name  59299 non-null  object
 5   language         59299 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.2+ MB


In [33]:
df_itis_tsn = df_merge[(df_merge["n_usage"] == "valid")]

### number of rows reduces to 59,299 (valid rows) from 152,250

In [35]:
df_itis_tsn.shape

(59299, 6)

#### change column name `complete_name` to `scientific_name` and `vernacular_name` to `common_name`

In [37]:
df_itis_tsn = df_itis_tsn.rename(columns={'complete_name':'scientific_name', 
                            'vernacular_name':'common_name'})
df_itis_tsn.head(3)

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
0,13757,Antilocapra americana peninsularis,valid,5,peninsular pronghorn,English
1,46861,Porifera,valid,5,sponges,English
2,47618,Hippospongia lachne,valid,5,sheepswool sponge,English


In [38]:
df_itis_tsn['scientific_name'] = df_itis_tsn['scientific_name'].str.strip().str.capitalize()
df_itis_tsn['common_name'] = df_itis_tsn['common_name'].str.strip().str.capitalize()

#### The same (tsn & scientific name) has multiple common names

In [40]:
df_itis_tsn[df_itis_tsn["tsn"]==161706]

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
8459,161706,Alosa pseudoharengus,valid,5,Alewife,English
8460,161706,Alosa pseudoharengus,valid,5,Kyak,English
8461,161706,Alosa pseudoharengus,valid,5,Bigeye herring,English
8462,161706,Alosa pseudoharengus,valid,5,Branch herring,English
8463,161706,Alosa pseudoharengus,valid,5,Freshwater herring,English
8464,161706,Alosa pseudoharengus,valid,5,Gray herring,English
8465,161706,Alosa pseudoharengus,valid,5,White herring,English
8466,161706,Alosa pseudoharengus,valid,5,Sawbelly,English
8467,161706,Alosa pseudoharengus,valid,5,Grayback,English


In [41]:
df_itis_tsn["tsn"].nunique()

47839

In [42]:
df_unique_tsn_scientific_name = df_itis_tsn.drop_duplicates(subset=["tsn"])
df_unique_tsn_scientific_name.shape

(47839, 6)

In [43]:
## as_index=False prevents the grouped columns from becoming the index
tsn_grpby = df_itis_tsn.groupby(["tsn"], as_index=False).agg({"common_name": '/ '.join})

In [44]:
tsn_grpby.head()

,tsn,common_name
0,13757,Peninsular pronghorn
1,46861,Sponges
2,47618,Sheepswool sponge
3,48738,Cnidarians/ Coelenterates
4,48739,Hydralike animals/ Hydroids/ Hydrozoans


In [45]:
tsn_grpby_df = tsn_grpby.set_index("tsn")
tsn_grpby_df.head(7)

,common_name
tsn,
13757,Peninsular pronghorn
46861,Sponges
47618,Sheepswool sponge
48738,Cnidarians/ Coelenterates
48739,Hydralike animals/ Hydroids/ Hydrozoans
48821,Orange hydroid
48891,Club hydroid


In [46]:
tsn_grpby_dict1 = tsn_grpby_df.to_dict()

In [47]:
tsn_grpby_dict1 = tsn_grpby_dict1["common_name"]  

In [48]:
# tsn_grpby_dict1

In [49]:
df_unique_tsn_scientific_name["common_name"] = \
        df_unique_tsn_scientific_name["tsn"].map(tsn_grpby_dict1) 

C:\Users\elias\AppData\Local\Temp\ipykernel_16368\3141572712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_tsn_scientific_name["common_name"] = \


In [50]:
df_unique_tsn_scientific_name.head()

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
0,13757,Antilocapra americana peninsularis,valid,5,Peninsular pronghorn,English
1,46861,Porifera,valid,5,Sponges,English
2,47618,Hippospongia lachne,valid,5,Sheepswool sponge,English
3,48738,Cnidaria,valid,5,Cnidarians/ Coelenterates,English
5,48739,Hydrozoa,valid,5,Hydralike animals/ Hydroids/ Hydrozoans,English


In [51]:
df_unique_tsn_scientific_name[df_unique_tsn_scientific_name["scientific_name"].str.contains("Alosa")]

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
8443,161701,Alosa,valid,5,River herrings,English
8444,161702,Alosa sapidissima,valid,5,American shad/ Atlantic shad/ Common shad/ White shad,English
8448,161703,Alosa aestivalis,valid,5,Blueback herring/ Blueback shad,English
8450,161704,Alosa mediocris,valid,5,Hickory shad/ Hickory jack/ Shad herring/ Freshwater taylor/ Fall herring/ Bonejack,English
8456,161705,Alosa alabamae,valid,5,Alabama shad/ Gulf shad/ Ohio shad,English
8459,161706,Alosa pseudoharengus,valid,5,Alewife/ Kyak/ Bigeye herring/ Branch herring/ Freshwater herring/ Gray herring/ White herring/ ...,English
8468,161707,Alosa chrysochloris,valid,5,Skipjack herring/ Skipjack shad/ Blue herring/ Green herring/ Golden shad/ River herring/ Skipjack,English
8475,161708,Alosa alosa,valid,5,Alice shad/ Allis shad,English
8477,161710,Alosa caspia,valid,5,Caspian shad,English
8478,161712,Alosa caspia nordmanni,valid,5,Danube shad,English


#### There are 47,839 valid unique tsn number with English common name

In [53]:
df_unique_tsn_scientific_name.tsn.nunique()

47839

#### Some test query

In [55]:
df_unique_tsn_scientific_name[df_unique_tsn_scientific_name["scientific_name"]=="Catostomus commersonii"]

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
24175,553273,Catostomus commersonii,valid,5,White sucker,English


In [56]:
df_unique_tsn_scientific_name[df_unique_tsn_scientific_name["tsn"]==161706]

,tsn,scientific_name,n_usage,kingdom_id,common_name,language
8459,161706,Alosa pseudoharengus,valid,5,Alewife/ Kyak/ Bigeye herring/ Branch herring/ Freshwater herring/ Gray herring/ White herring/ ...,English


In [57]:
df_unique_tsn_scientific_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47839 entries, 0 to 59298
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tsn              47839 non-null  int64 
 1   scientific_name  47839 non-null  object
 2   n_usage          47839 non-null  object
 3   kingdom_id       47839 non-null  int64 
 4   common_name      47839 non-null  object
 5   language         47839 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


In [58]:
df_itis_tsn = df_unique_tsn_scientific_name[["tsn", "scientific_name", "common_name"]]

In [59]:
df_itis_tsn.head()

,tsn,scientific_name,common_name
0,13757,Antilocapra americana peninsularis,Peninsular pronghorn
1,46861,Porifera,Sponges
2,47618,Hippospongia lachne,Sheepswool sponge
3,48738,Cnidaria,Cnidarians/ Coelenterates
5,48739,Hydrozoa,Hydralike animals/ Hydroids/ Hydrozoans


#### save the original TSN data

In [61]:
# import pickle
# filename = 'TSN.pkl'
# with open(filename, 'wb') as file:
#     pickle.dump(df_itis_tsn, file)

#### Can be saved as `parquet` file format, which is simpler to read/write in pandas

In [63]:
df_itis_tsn.to_parquet("itis_tsn.parquet", index=False)
# read_df = pd.read_parquet("itis_tsn.parquet")


In [64]:
# read_df = pd.read_parquet("itis_tsn.parquet")

In [65]:
# with open(filename, 'rb') as file:
#     original_tsn_df =pickle.load(file)
# print(original_tsn_df.shape)
# print(type(original_tsn_df))
# print(original_tsn_df.head())